## Crypto Sentiment Analyzer

In [12]:
# Imports
import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
%matplotlib inline

In [13]:
load_dotenv()

True

## Establish Baseline Performance

#### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [14]:
# Import the OHLCV dataset into a 

btc_url = 'https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD'
eth_url = 'https://api.alternative.me/v2/ticker/Ethereum/?convert=USD'
bnb_url = 'https://api.alternative.me/v2/ticker/BinanceCoin/?convert=USD'
xrp_url = 'https://api.alternative.me/v2/ticker/ripple/?convert=USD'
luna_url = 'https://api.alternative.me/v2/ticker/terra-luna/?convert=USD'
cardano_url ='https://api.alternative.me/v2/ticker/cardano/?convert=USD'
avax_url = 'https://api.alternative.me/v2/ticker/avalanche-2/?convert=USD'
dot_url = 'https://api.alternative.me/v2/ticker/polkadot/?convert=USD'
doge_url = 'https://api.alternative.me/v2/ticker/doge/?convert=USD'
shiba_url= 'https://api.alternative.me/v2/ticker/shiba-inu/?convert=USD'
api_key = os.getenv('API_Key')

In [4]:
btc_response = requests.get(btc_url).json()
eth_response= requests.get(eth_url).json()
bnb_response= requests.get(bnb_url).json()
xrp_response= requests.get(xrp_url).json()
luna_response = requests.get(luna_url).json()
cardano_response = requests.get(cardano_url).json()
avax_response = requests.get(avax_url).json()
dot_response= requests.get(dot_url).json()
doge_response = requests.get(doge_url).json()
shiba_response = requests.get(shiba_url).json()



In [15]:
alpaca_api_key= os.getenv('API_Key')
alpaca_secret_key = os.getenv('API_Secret_Key')


alpaca= tradeapi.REST(
alpaca_api_key,
alpaca_secret_key,
api_version= "v2")

In [16]:

# Filter the date index and close columns
tickers = ['BTC']
# Use the pct_change function to generate returns from close prices
timeframe = '1D'

limit_rows= 1000
# Drop all NaN values from the DataFrame

start_date = pd.Timestamp('2018-01-01', tz= "America/New_York").isoformat()
end_date = pd.Timestamp('2022-03-15', tz="America/New_York").isoformat()


In [20]:
start_date

'2018-01-01T00:00:00-05:00'

In [26]:
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
prices_df = alpaca.get_bars(
tickers,
# timeframe,
TimeFrame(1, TimeFrameUnit.Day), 
# start=start_date,
# end=end_date,
start='2018-01-01', 
end='2022-03-15',
limit=limit_rows).df

prices_df.head(100)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2021-04-16 04:00:00+00:00,100.000,100.8900,98.7700,99.2447,4497,186,100.273227,BTC
2021-04-19 04:00:00+00:00,99.130,99.1300,98.9850,99.1012,4551,96,99.104105,BTC
2021-04-20 04:00:00+00:00,98.790,99.3000,98.7900,99.0441,3736,82,98.932931,BTC
2021-04-21 04:00:00+00:00,99.030,99.0100,98.9000,98.9705,997,42,98.950429,BTC
2021-04-22 04:00:00+00:00,98.950,99.0400,98.8600,99.0010,1599,68,98.958802,BTC
2021-04-23 04:00:00+00:00,99.070,99.0100,98.8801,98.9800,1903,42,98.978855,BTC
2021-04-26 04:00:00+00:00,99.000,99.0000,98.9082,98.9700,2503,77,98.956571,BTC
2021-04-27 04:00:00+00:00,98.950,98.9500,98.8200,98.8326,1317,41,98.901892,BTC
2021-04-28 04:00:00+00:00,98.880,98.8864,98.7600,98.8864,1001,58,98.833692,BTC


#### Step 2: Generate trading signals using short- and long-window SMA values.

 #### Step 3: Split the data into training and testing datasets.

#### Step 4: Use the SVC classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

#### Step 5: Review the classification report associated with the SVC model predictions.

#### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

#### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.



#### Tune the Baseline Trading Algorithm


## Evaluate a New Machine Learning Classifier

#### Step 1: Import a new classifier, such as AdaBoost, DecisionTreeClassifier, or LogisticRegression. (For the full list of classifiers, refer to the Supervised learning page in the scikit-learn documentation.)

#### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

#### Step 3: Backtest the new model to evaluate its performance.